# HYPERPARAMETER TUNING 

## Loading the Necessary Libraries 

In [1]:
# !pip install optuna

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.model
import optuna
from catboost import CatBoostClassifier as cat
import lightgbm as lgb
RANDOM_SEED = 42

C:\Users\DEV\Anaconda3\lib\site-packages\distributed\utils.py:133: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [WinError 10065] A socket operation was attempted to an unreachable host
  RuntimeWarning,


In [3]:
df = pd.read_csv("NewTrain.csv")
test = pd.read_csv("NewTest.csv")
sample_submission = pd.read_csv("sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "song_popularity", "kfold")]
test = test[useful_features]


In [4]:
#Since the columns are of different size we are scaling them
lE = preprocessing.RobustScaler()
df[useful_features] = lE.fit_transform(df[useful_features])
test[useful_features] = lE.transform(test[useful_features])

In [5]:
df.kfold.value_counts()

9    4000
8    4000
7    4000
6    4000
5    4000
4    4000
3    4000
2    4000
1    4000
0    4000
Name: kfold, dtype: int64

# BUILDING_MODELS 

In [13]:
cat_params = {
            'depth' : 4,
            'grow_policy' : 'Depthwise',
            'l2_leaf_reg' : 97.2237331816313,
            'random_strength' : 0.5686428156275426,
            'learning_rate' : 0.012567270456139758,
            'iterations' : 10000,
            'loss_function' : 'Logloss',
            'bagging_temperature': 0.6051472546289562,
            'border_count': 202,
            'eval_metric' : 'AUC',
            'use_best_model' : True,
            'early_stopping_rounds' : 500,
            'task_type' : 'CPU',
            'thread_count' : 4,
            'verbose' : False
    
}
        
        
    
lgbm_params = {
              'learning_rate': 0.0027109050113455304,
              'n_estimators': 11348,
              'reg_lambda': 0.5106388532860763,
              'reg_alpha': 0.27004521552837574,
              'max_depth': 130,
              'num_leaves': 44,
              'feature_fraction': 0.4263819349686964,
              'bagging_fraction': 0.49593550461680574,
              'pos_bagging_fraction': 0.5371623198828565,
              'neg_bagging_fraction': 0.30616770109746205,
              'bagging_freq': 6,
              'min_child_samples': 89,
              'objective': 'binary',
              'metric': 'auc',
              'early_stopping_round': 200,
              'verbosity': -1,
              'n_jobs': 4,
              'random_state':RANDOM_SEED
}

lgbm_params1 = {
            'n_estimators': 20000,
            'learning_rate': 5e-3,
            'subsample': 0.6,
            'subsample_freq': 1,
            'colsample_bytree': 0.4,
            'reg_alpha': 10.0,
            'reg_lambda': 1e-1,
            'min_child_weight': 256,
            'min_child_samples': 20,
            'early_stopping_round': 200,
            'objective': 'binary',
            'metric': 'auc',
            'verbosity': -1,
            'n_jobs': 4,
            'random_state': RANDOM_SEED
}
        
lgbm_params2 = {
              'task': 'train',
              'boosting_type': 'gbdt',
              'objective': 'binary',
              'metric': 'auc',
              'early_stopping_round': 200,
              'learning_rate': 0.001635,
              'max_depth': 3,
              'feature_fraction': 0.2256038826485174,
              'bagging_fraction': 0.7705303688019942,
              'min_child_samples': 290,
              'reg_alpha': 14.68267919457715,
              'reg_lambda': 66.156,
              'max_bin': 772,
              'min_data_per_group': 177,
              'bagging_freq': 1,
              'cat_smooth': 96,
              'cat_l2': 17,
              'verbosity': -1,
              'random_state': RANDOM_SEED,
              'n_estimators': 10000,
}

## CATBOOST 

In [7]:
final_test_preds=[]
final_valid_preds={}
scores=[]

for fold in range(10):
    xtrain=df[df.kfold != fold].reset_index(drop=True)
    xvalid=df[df.kfold == fold].reset_index(drop=True)
    xtest=test.copy()
    
    valid_id=xvalid.id.values.tolist()
    
    ytrain=xtrain.song_popularity
    yvalid=xvalid.song_popularity
    
    xtrain=xtrain[useful_features]
    xvalid=xvalid[useful_features]

    
    model = cat(**cat_params)
    model.fit(xtrain, ytrain,eval_set=[(xvalid,yvalid)])
    
    preds_valid=model.predict_proba(xvalid)[:,1]
    preds_test=model.predict_proba(xtest[useful_features])[:,1]
    
    
    final_test_preds.append(preds_test)
    final_valid_preds.update(dict(zip(valid_id,preds_valid)))
    
    roc1= roc_auc_score(yvalid,preds_valid)
    #Score 
    scores.append(roc1)
    print(f"fold|split:{fold},roc:{roc1}")
    
    
print(np.mean(scores))
final_valid_pred1=pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
final_valid_pred1.columns=['id',"preds_1"]
final_valid_pred1.to_csv('train_pred_1.csv',index=False)
    
y=np.mean(np.column_stack(final_test_preds),axis=1)
test_preds1=pd.DataFrame(y,columns=['test_preds1'])
test_preds1['id']=test_preds1.index
test_preds1.columns=['preds_1','id']
test_preds1.to_csv("test_preds1.csv",index=False)
    

fold|split:0,roc:0.5846054707165632
fold|split:1,roc:0.5836587308525414
fold|split:2,roc:0.5727489323222812
fold|split:3,roc:0.5584590799253262
fold|split:4,roc:0.5703020662112184
fold|split:5,roc:0.5839670820179694
fold|split:6,roc:0.5897193488840409
fold|split:7,roc:0.5647817474157053
fold|split:8,roc:0.5736580702923575
fold|split:9,roc:0.5744547343203028
0.5756355262958307


In [8]:
final_test_preds=[]
final_valid_preds={}
scores=[]

for fold in range(10):
    xtrain=df[df.kfold != fold].reset_index(drop=True)
    xvalid=df[df.kfold == fold].reset_index(drop=True)
    xtest=test.copy()
    
    valid_id=xvalid.id.values.tolist()
    
    ytrain=xtrain.song_popularity
    yvalid=xvalid.song_popularity
    
    xtrain=xtrain[useful_features]
    xvalid=xvalid[useful_features]

    
    model = cat(n_estimators=5000, max_depth=6, eval_metric='AUC', reg_lambda = 370,random_state=890)
    model.fit(xtrain, ytrain,eval_set=[(xvalid,yvalid)],early_stopping_rounds=500,use_best_model=True)
    
    preds_valid=model.predict_proba(xvalid)[:,1]
    preds_test=model.predict_proba(xtest[useful_features])[:,1]
    
    
    final_test_preds.append(preds_test)
    final_valid_preds.update(dict(zip(valid_id,preds_valid)))
    
    roc1= roc_auc_score(yvalid,preds_valid)
    #Score 
    scores.append(roc1)
    print(f"fold|split:{fold},roc:{roc1}")
    
    
print(np.mean(scores))
final_valid_pred1=pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
final_valid_pred1.columns=['id',"preds_4"]
final_valid_pred1.to_csv('train_pred_4.csv',index=False)
    
y=np.mean(np.column_stack(final_test_preds),axis=1)
test_preds1=pd.DataFrame(y,columns=['test_preds4'])
test_preds1['id']=test_preds1.index
test_preds1.columns=['preds_4','id']
test_preds1.to_csv("test_preds4.csv",index=False)
    

0:	test: 0.5487209	best: 0.5487209 (0)	total: 66ms	remaining: 5m 29s
1:	test: 0.5575875	best: 0.5575875 (1)	total: 102ms	remaining: 4m 15s
2:	test: 0.5614188	best: 0.5614188 (2)	total: 145ms	remaining: 4m 2s
3:	test: 0.5600545	best: 0.5614188 (2)	total: 186ms	remaining: 3m 51s
4:	test: 0.5569258	best: 0.5614188 (2)	total: 240ms	remaining: 3m 59s
5:	test: 0.5554403	best: 0.5614188 (2)	total: 277ms	remaining: 3m 50s
6:	test: 0.5567431	best: 0.5614188 (2)	total: 310ms	remaining: 3m 41s
7:	test: 0.5563734	best: 0.5614188 (2)	total: 342ms	remaining: 3m 33s
8:	test: 0.5566025	best: 0.5614188 (2)	total: 376ms	remaining: 3m 28s
9:	test: 0.5575561	best: 0.5614188 (2)	total: 409ms	remaining: 3m 23s
10:	test: 0.5557401	best: 0.5614188 (2)	total: 440ms	remaining: 3m 19s
11:	test: 0.5561965	best: 0.5614188 (2)	total: 473ms	remaining: 3m 16s
12:	test: 0.5568737	best: 0.5614188 (2)	total: 506ms	remaining: 3m 14s
13:	test: 0.5565265	best: 0.5614188 (2)	total: 539ms	remaining: 3m 11s
14:	test: 0.555093

116:	test: 0.5670519	best: 0.5670549 (115)	total: 4.36s	remaining: 3m 1s
117:	test: 0.5674576	best: 0.5674576 (117)	total: 4.39s	remaining: 3m 1s
118:	test: 0.5675358	best: 0.5675358 (118)	total: 4.44s	remaining: 3m 2s
119:	test: 0.5677014	best: 0.5677014 (119)	total: 4.48s	remaining: 3m 2s
120:	test: 0.5677371	best: 0.5677371 (120)	total: 4.53s	remaining: 3m 2s
121:	test: 0.5675755	best: 0.5677371 (120)	total: 4.57s	remaining: 3m 2s
122:	test: 0.5675701	best: 0.5677371 (120)	total: 4.61s	remaining: 3m 2s
123:	test: 0.5678010	best: 0.5678010 (123)	total: 4.65s	remaining: 3m 2s
124:	test: 0.5678078	best: 0.5678078 (124)	total: 4.69s	remaining: 3m 2s
125:	test: 0.5678902	best: 0.5678902 (125)	total: 4.72s	remaining: 3m 2s
126:	test: 0.5676534	best: 0.5678902 (125)	total: 4.75s	remaining: 3m 2s
127:	test: 0.5678338	best: 0.5678902 (125)	total: 4.78s	remaining: 3m 2s
128:	test: 0.5679535	best: 0.5679535 (128)	total: 4.82s	remaining: 3m 1s
129:	test: 0.5680703	best: 0.5680703 (129)	total: 4

228:	test: 0.5738455	best: 0.5738455 (228)	total: 7.99s	remaining: 2m 46s
229:	test: 0.5738026	best: 0.5738455 (228)	total: 8.02s	remaining: 2m 46s
230:	test: 0.5737191	best: 0.5738455 (228)	total: 8.06s	remaining: 2m 46s
231:	test: 0.5735577	best: 0.5738455 (228)	total: 8.09s	remaining: 2m 46s
232:	test: 0.5736125	best: 0.5738455 (228)	total: 8.12s	remaining: 2m 46s
233:	test: 0.5735746	best: 0.5738455 (228)	total: 8.15s	remaining: 2m 45s
234:	test: 0.5734769	best: 0.5738455 (228)	total: 8.18s	remaining: 2m 45s
235:	test: 0.5737094	best: 0.5738455 (228)	total: 8.21s	remaining: 2m 45s
236:	test: 0.5737269	best: 0.5738455 (228)	total: 8.24s	remaining: 2m 45s
237:	test: 0.5738928	best: 0.5738928 (237)	total: 8.28s	remaining: 2m 45s
238:	test: 0.5741323	best: 0.5741323 (238)	total: 8.31s	remaining: 2m 45s
239:	test: 0.5741234	best: 0.5741323 (238)	total: 8.34s	remaining: 2m 45s
240:	test: 0.5741597	best: 0.5741597 (240)	total: 8.37s	remaining: 2m 45s
241:	test: 0.5742005	best: 0.5742005 (

339:	test: 0.5775103	best: 0.5776591 (338)	total: 11.6s	remaining: 2m 38s
340:	test: 0.5775391	best: 0.5776591 (338)	total: 11.6s	remaining: 2m 38s
341:	test: 0.5774824	best: 0.5776591 (338)	total: 11.6s	remaining: 2m 38s
342:	test: 0.5776113	best: 0.5776591 (338)	total: 11.6s	remaining: 2m 38s
343:	test: 0.5776158	best: 0.5776591 (338)	total: 11.7s	remaining: 2m 38s
344:	test: 0.5777259	best: 0.5777259 (344)	total: 11.7s	remaining: 2m 37s
345:	test: 0.5777737	best: 0.5777737 (345)	total: 11.7s	remaining: 2m 37s
346:	test: 0.5777442	best: 0.5777737 (345)	total: 11.8s	remaining: 2m 37s
347:	test: 0.5778183	best: 0.5778183 (347)	total: 11.8s	remaining: 2m 37s
348:	test: 0.5777332	best: 0.5778183 (347)	total: 11.8s	remaining: 2m 37s
349:	test: 0.5778494	best: 0.5778494 (349)	total: 11.9s	remaining: 2m 37s
350:	test: 0.5777936	best: 0.5778494 (349)	total: 11.9s	remaining: 2m 37s
351:	test: 0.5779837	best: 0.5779837 (351)	total: 11.9s	remaining: 2m 37s
352:	test: 0.5781671	best: 0.5781671 (

450:	test: 0.5804053	best: 0.5804808 (448)	total: 15s	remaining: 2m 31s
451:	test: 0.5804542	best: 0.5804808 (448)	total: 15s	remaining: 2m 31s
452:	test: 0.5803785	best: 0.5804808 (448)	total: 15.1s	remaining: 2m 31s
453:	test: 0.5802963	best: 0.5804808 (448)	total: 15.1s	remaining: 2m 31s
454:	test: 0.5803202	best: 0.5804808 (448)	total: 15.1s	remaining: 2m 31s
455:	test: 0.5804399	best: 0.5804808 (448)	total: 15.2s	remaining: 2m 31s
456:	test: 0.5805033	best: 0.5805033 (456)	total: 15.2s	remaining: 2m 30s
457:	test: 0.5805441	best: 0.5805441 (457)	total: 15.2s	remaining: 2m 30s
458:	test: 0.5804211	best: 0.5805441 (457)	total: 15.2s	remaining: 2m 30s
459:	test: 0.5805200	best: 0.5805441 (457)	total: 15.3s	remaining: 2m 30s
460:	test: 0.5808303	best: 0.5808303 (460)	total: 15.3s	remaining: 2m 30s
461:	test: 0.5807917	best: 0.5808303 (460)	total: 15.4s	remaining: 2m 30s
462:	test: 0.5808725	best: 0.5808725 (462)	total: 15.4s	remaining: 2m 30s
463:	test: 0.5808781	best: 0.5808781 (463)

...

187:	test: 0.5767101	best: 0.5768014 (186)	total: 6.61s	remaining: 2m 49s
188:	test: 0.5767103	best: 0.5768014 (186)	total: 6.64s	remaining: 2m 49s
189:	test: 0.5767737	best: 0.5768014 (186)	total: 6.67s	remaining: 2m 48s
190:	test: 0.5767613	best: 0.5768014 (186)	total: 6.71s	remaining: 2m 48s
191:	test: 0.5769752	best: 0.5769752 (191)	total: 6.74s	remaining: 2m 48s
192:	test: 0.5770299	best: 0.5770299 (192)	total: 6.77s	remaining: 2m 48s
193:	test: 0.5768860	best: 0.5770299 (192)	total: 6.81s	remaining: 2m 48s
194:	test: 0.5768971	best: 0.5770299 (192)	total: 6.84s	remaining: 2m 48s
195:	test: 0.5771032	best: 0.5771032 (195)	total: 6.87s	remaining: 2m 48s
196:	test: 0.5768122	best: 0.5771032 (195)	total: 6.91s	remaining: 2m 48s
197:	test: 0.5768378	best: 0.5771032 (195)	total: 6.94s	remaining: 2m 48s
198:	test: 0.5767936	best: 0.5771032 (195)	total: 6.97s	remaining: 2m 48s
199:	test: 0.5767761	best: 0.5771032 (195)	total: 7s	remaining: 2m 48s
200:	test: 0.5767505	best: 0.5771032 (195

298:	test: 0.5759266	best: 0.5771032 (195)	total: 10.2s	remaining: 2m 39s
299:	test: 0.5760333	best: 0.5771032 (195)	total: 10.2s	remaining: 2m 39s
300:	test: 0.5760249	best: 0.5771032 (195)	total: 10.2s	remaining: 2m 39s
301:	test: 0.5760608	best: 0.5771032 (195)	total: 10.3s	remaining: 2m 39s
302:	test: 0.5761120	best: 0.5771032 (195)	total: 10.3s	remaining: 2m 39s
303:	test: 0.5761125	best: 0.5771032 (195)	total: 10.3s	remaining: 2m 39s
304:	test: 0.5761669	best: 0.5771032 (195)	total: 10.3s	remaining: 2m 39s
305:	test: 0.5760953	best: 0.5771032 (195)	total: 10.4s	remaining: 2m 39s
306:	test: 0.5762213	best: 0.5771032 (195)	total: 10.4s	remaining: 2m 39s
307:	test: 0.5763404	best: 0.5771032 (195)	total: 10.4s	remaining: 2m 39s
308:	test: 0.5762044	best: 0.5771032 (195)	total: 10.5s	remaining: 2m 39s
309:	test: 0.5760802	best: 0.5771032 (195)	total: 10.5s	remaining: 2m 38s
310:	test: 0.5760438	best: 0.5771032 (195)	total: 10.5s	remaining: 2m 38s
311:	test: 0.5759721	best: 0.5771032 (

409:	test: 0.5744437	best: 0.5771032 (195)	total: 13.6s	remaining: 2m 32s
410:	test: 0.5744639	best: 0.5771032 (195)	total: 13.7s	remaining: 2m 32s
411:	test: 0.5744488	best: 0.5771032 (195)	total: 13.7s	remaining: 2m 32s
412:	test: 0.5745420	best: 0.5771032 (195)	total: 13.7s	remaining: 2m 32s
413:	test: 0.5746506	best: 0.5771032 (195)	total: 13.8s	remaining: 2m 32s
414:	test: 0.5746792	best: 0.5771032 (195)	total: 13.8s	remaining: 2m 32s
415:	test: 0.5746969	best: 0.5771032 (195)	total: 13.8s	remaining: 2m 32s
416:	test: 0.5746250	best: 0.5771032 (195)	total: 13.9s	remaining: 2m 32s
417:	test: 0.5746555	best: 0.5771032 (195)	total: 13.9s	remaining: 2m 32s
418:	test: 0.5747376	best: 0.5771032 (195)	total: 13.9s	remaining: 2m 32s
419:	test: 0.5745965	best: 0.5771032 (195)	total: 13.9s	remaining: 2m 32s
420:	test: 0.5745717	best: 0.5771032 (195)	total: 14s	remaining: 2m 32s
421:	test: 0.5746107	best: 0.5771032 (195)	total: 14s	remaining: 2m 32s
422:	test: 0.5746051	best: 0.5771032 (195)

521:	test: 0.5738103	best: 0.5771032 (195)	total: 17.5s	remaining: 2m 29s
522:	test: 0.5738184	best: 0.5771032 (195)	total: 17.5s	remaining: 2m 29s
523:	test: 0.5738895	best: 0.5771032 (195)	total: 17.5s	remaining: 2m 29s
524:	test: 0.5739620	best: 0.5771032 (195)	total: 17.6s	remaining: 2m 29s
525:	test: 0.5741427	best: 0.5771032 (195)	total: 17.6s	remaining: 2m 30s
526:	test: 0.5743497	best: 0.5771032 (195)	total: 17.7s	remaining: 2m 30s
527:	test: 0.5743828	best: 0.5771032 (195)	total: 17.7s	remaining: 2m 30s
528:	test: 0.5743343	best: 0.5771032 (195)	total: 17.8s	remaining: 2m 30s
529:	test: 0.5742155	best: 0.5771032 (195)	total: 17.8s	remaining: 2m 30s
530:	test: 0.5741339	best: 0.5771032 (195)	total: 17.8s	remaining: 2m 30s
531:	test: 0.5741543	best: 0.5771032 (195)	total: 17.9s	remaining: 2m 30s
532:	test: 0.5741969	best: 0.5771032 (195)	total: 17.9s	remaining: 2m 30s
533:	test: 0.5743009	best: 0.5771032 (195)	total: 17.9s	remaining: 2m 30s
534:	test: 0.5741821	best: 0.5771032 (

633:	test: 0.5727681	best: 0.5771032 (195)	total: 21.1s	remaining: 2m 25s
634:	test: 0.5727143	best: 0.5771032 (195)	total: 21.1s	remaining: 2m 25s
635:	test: 0.5727743	best: 0.5771032 (195)	total: 21.1s	remaining: 2m 25s
636:	test: 0.5726315	best: 0.5771032 (195)	total: 21.2s	remaining: 2m 24s
637:	test: 0.5726660	best: 0.5771032 (195)	total: 21.2s	remaining: 2m 24s
638:	test: 0.5726248	best: 0.5771032 (195)	total: 21.2s	remaining: 2m 24s
639:	test: 0.5725100	best: 0.5771032 (195)	total: 21.3s	remaining: 2m 24s
640:	test: 0.5724209	best: 0.5771032 (195)	total: 21.3s	remaining: 2m 24s
641:	test: 0.5723613	best: 0.5771032 (195)	total: 21.3s	remaining: 2m 24s
642:	test: 0.5723196	best: 0.5771032 (195)	total: 21.4s	remaining: 2m 24s
643:	test: 0.5722751	best: 0.5771032 (195)	total: 21.4s	remaining: 2m 24s
644:	test: 0.5722646	best: 0.5771032 (195)	total: 21.4s	remaining: 2m 24s
645:	test: 0.5723446	best: 0.5771032 (195)	total: 21.5s	remaining: 2m 24s
646:	test: 0.5723465	best: 0.5771032 (

In [9]:
final_test_preds=[]
final_valid_preds={}
scores=[]

for fold in range(10):
    xtrain=df[df.kfold != fold].reset_index(drop=True)
    xvalid=df[df.kfold == fold].reset_index(drop=True)
    xtest=test.copy()
    
    valid_id=xvalid.id.values.tolist()
    
    ytrain=xtrain.song_popularity
    yvalid=xvalid.song_popularity
    
    xtrain=xtrain[useful_features]
    xvalid=xvalid[useful_features]

    
    model = cat(bootstrap_type='Bernoulli', 
                              iterations=5000, silent=True, eval_metric='AUC',random_state=456)
    model.fit(xtrain, ytrain,eval_set=[(xvalid,yvalid)],early_stopping_rounds=500,use_best_model=True)
    
    preds_valid=model.predict_proba(xvalid)[:,1]
    preds_test=model.predict_proba(xtest[useful_features])[:,1]
    
    
    final_test_preds.append(preds_test)
    final_valid_preds.update(dict(zip(valid_id,preds_valid)))
    
    roc1= roc_auc_score(yvalid,preds_valid)
    #Score 
    scores.append(roc1)
    print(f"fold|split:{fold},roc:{roc1}")
    
    
print(np.mean(scores))
final_valid_pred1=pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
final_valid_pred1.columns=['id',"preds_5"]
final_valid_pred1.to_csv('train_pred_5.csv',index=False)
    
y=np.mean(np.column_stack(final_test_preds),axis=1)
test_preds1=pd.DataFrame(y,columns=['test_preds5'])
test_preds1['id']=test_preds1.index
test_preds1.columns=['preds_5','id']
test_preds1.to_csv("test_preds5.csv",index=False)
    

fold|split:0,roc:0.582577747944286
fold|split:1,roc:0.5841732690524835
fold|split:2,roc:0.5694525685485774
fold|split:3,roc:0.5602787616023449
fold|split:4,roc:0.5710829951715636
fold|split:5,roc:0.5816422549841748
fold|split:6,roc:0.5909372713542541
fold|split:7,roc:0.5627889994301836
fold|split:8,roc:0.5754311987128999
fold|split:9,roc:0.5717534959698312
0.5750118562770601


## LIGHTGBM 

In [10]:
final_test_preds=[]
final_valid_preds={}
scores=[]

for fold in range(10):
    xtrain=df[df.kfold != fold].reset_index(drop=True)
    xvalid=df[df.kfold == fold].reset_index(drop=True)
    xtest=test.copy()
    
    valid_id=xvalid.id.values.tolist()
    
    ytrain=xtrain.song_popularity
    yvalid=xvalid.song_popularity
    
    xtrain=xtrain[useful_features]
    xvalid=xvalid[useful_features]

    
    model = lgb.LGBMClassifier(**lgbm_params)
    model.fit(xtrain, ytrain,eval_set=[(xvalid, yvalid)])
    
    preds_valid=model.predict_proba(xvalid)[:,1]
    preds_test=model.predict_proba(xtest[useful_features])[:,1]
    
    
    final_test_preds.append(preds_test)
    final_valid_preds.update(dict(zip(valid_id,preds_valid)))
    
    roc1= roc_auc_score(yvalid,preds_valid)
    #Score 
    scores.append(roc1)
    print(f"fold|split:{fold},roc:{roc1}")
    
    
print(np.mean(scores))
final_valid_pred1=pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
final_valid_pred1.columns=['id',"preds_2"]
final_valid_pred1.to_csv('train_pred_2.csv',index=False)
    
y=np.mean(np.column_stack(final_test_preds),axis=1)
test_preds1=pd.DataFrame(y,columns=['test_preds2'])
test_preds1['id']=test_preds1.index
test_preds1.columns=['preds_2','id']
test_preds1.to_csv("test_preds2.csv",index=False)
    

[LightGBM] [Warning] feature_fraction is set=0.4263819349686964, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4263819349686964
[LightGBM] [Warning] bagging_fraction is set=0.49593550461680574, subsample=1.0 will be ignored. Current value: bagging_fraction=0.49593550461680574
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[1]	valid_0's auc: 0.526115
[2]	valid_0's auc: 0.537562
[3]	valid_0's auc: 0.539653
[4]	valid_0's auc: 0.549623
[5]	valid_0's auc: 0.55163
[6]	valid_0's auc: 0.552595
[7]	valid_0's auc: 0.554391
[8]	valid_0's auc: 0.557091
[9]	valid_0's auc: 0.556694
[10]	valid_0's auc: 0.559579
[11]	valid_0's auc: 0.559262
[12]	valid_0's auc: 0.559557
[13]	valid_0's auc: 0.560056
[14]	valid_0's auc: 0.560252
[15]	valid_0's auc: 0.560108
[16]	valid_0's auc: 0.559944
[17]	valid_0's auc: 0.560861
[18]	valid_0's auc: 0.56149
[19]	valid_0's auc: 0.563916
[20]	valid_0's auc: 0.564733
[21]	valid_0's auc: 

[275]	valid_0's auc: 0.585348
[276]	valid_0's auc: 0.585257
[277]	valid_0's auc: 0.585263
[278]	valid_0's auc: 0.585369
[279]	valid_0's auc: 0.585305
[280]	valid_0's auc: 0.585285
[281]	valid_0's auc: 0.585205
[282]	valid_0's auc: 0.585112
[283]	valid_0's auc: 0.58504
[284]	valid_0's auc: 0.585109
[285]	valid_0's auc: 0.585
[286]	valid_0's auc: 0.585038
[287]	valid_0's auc: 0.585026
[288]	valid_0's auc: 0.585145
[289]	valid_0's auc: 0.58498
[290]	valid_0's auc: 0.584994
[291]	valid_0's auc: 0.585068
[292]	valid_0's auc: 0.585025
[293]	valid_0's auc: 0.584917
[294]	valid_0's auc: 0.58504
[295]	valid_0's auc: 0.585008
[296]	valid_0's auc: 0.585206
[297]	valid_0's auc: 0.58507
[298]	valid_0's auc: 0.585035
[299]	valid_0's auc: 0.585069
[300]	valid_0's auc: 0.58499
[301]	valid_0's auc: 0.584752
[302]	valid_0's auc: 0.584632
[303]	valid_0's auc: 0.584719
[304]	valid_0's auc: 0.584862
[305]	valid_0's auc: 0.58491
[306]	valid_0's auc: 0.584937
[307]	valid_0's auc: 0.584988
[308]	valid_0's auc

[501]	valid_0's auc: 0.565323
[502]	valid_0's auc: 0.565348
[503]	valid_0's auc: 0.565341
[504]	valid_0's auc: 0.565321
[505]	valid_0's auc: 0.565256
[506]	valid_0's auc: 0.565246
[507]	valid_0's auc: 0.565228
[508]	valid_0's auc: 0.565324
[509]	valid_0's auc: 0.565288
[510]	valid_0's auc: 0.565271
[511]	valid_0's auc: 0.565196
[512]	valid_0's auc: 0.5653
[513]	valid_0's auc: 0.565325
[514]	valid_0's auc: 0.565321
[515]	valid_0's auc: 0.565313
[516]	valid_0's auc: 0.565329
[517]	valid_0's auc: 0.565327
[518]	valid_0's auc: 0.565333
[519]	valid_0's auc: 0.565357
[520]	valid_0's auc: 0.565331
[521]	valid_0's auc: 0.565359
[522]	valid_0's auc: 0.565323
[523]	valid_0's auc: 0.565278
[524]	valid_0's auc: 0.565265
[525]	valid_0's auc: 0.565324
[526]	valid_0's auc: 0.565354
[527]	valid_0's auc: 0.565327
[528]	valid_0's auc: 0.565393
[529]	valid_0's auc: 0.565384
[530]	valid_0's auc: 0.565355
[531]	valid_0's auc: 0.565419
[532]	valid_0's auc: 0.565467
[533]	valid_0's auc: 0.56538
[534]	valid_0

[237]	valid_0's auc: 0.553529
[238]	valid_0's auc: 0.553577
[239]	valid_0's auc: 0.553504
[240]	valid_0's auc: 0.553537
[241]	valid_0's auc: 0.55357
[242]	valid_0's auc: 0.553644
[243]	valid_0's auc: 0.553487
[244]	valid_0's auc: 0.553643
[245]	valid_0's auc: 0.553603
[246]	valid_0's auc: 0.553663
[247]	valid_0's auc: 0.553724
[248]	valid_0's auc: 0.553578
[249]	valid_0's auc: 0.553561
[250]	valid_0's auc: 0.553498
[251]	valid_0's auc: 0.55348
[252]	valid_0's auc: 0.553334
[253]	valid_0's auc: 0.553134
[254]	valid_0's auc: 0.553297
[255]	valid_0's auc: 0.553279
[256]	valid_0's auc: 0.553204
[257]	valid_0's auc: 0.553114
[258]	valid_0's auc: 0.553013
[259]	valid_0's auc: 0.553073
[260]	valid_0's auc: 0.553133
[261]	valid_0's auc: 0.553207
[262]	valid_0's auc: 0.553252
[263]	valid_0's auc: 0.55324
[264]	valid_0's auc: 0.553225
[265]	valid_0's auc: 0.553237
[266]	valid_0's auc: 0.55337
[267]	valid_0's auc: 0.553406
[268]	valid_0's auc: 0.55361
[269]	valid_0's auc: 0.553625
[270]	valid_0's

[784]	valid_0's auc: 0.556404
[785]	valid_0's auc: 0.556326
[786]	valid_0's auc: 0.556293
[787]	valid_0's auc: 0.556305
[788]	valid_0's auc: 0.556244
[789]	valid_0's auc: 0.556256
[790]	valid_0's auc: 0.556284
[791]	valid_0's auc: 0.556311
[792]	valid_0's auc: 0.556319
[793]	valid_0's auc: 0.556377
[794]	valid_0's auc: 0.556484
[795]	valid_0's auc: 0.556516
[796]	valid_0's auc: 0.556599
[797]	valid_0's auc: 0.556683
[798]	valid_0's auc: 0.556665
[799]	valid_0's auc: 0.556597
[800]	valid_0's auc: 0.556569
[801]	valid_0's auc: 0.556482
[802]	valid_0's auc: 0.55647
[803]	valid_0's auc: 0.556476
[804]	valid_0's auc: 0.556497
[805]	valid_0's auc: 0.556462
[806]	valid_0's auc: 0.556393
[807]	valid_0's auc: 0.556436
[808]	valid_0's auc: 0.556483
[809]	valid_0's auc: 0.556426
[810]	valid_0's auc: 0.556402
[811]	valid_0's auc: 0.556384
[812]	valid_0's auc: 0.55638
[813]	valid_0's auc: 0.556402
[814]	valid_0's auc: 0.55642
[815]	valid_0's auc: 0.556369
[816]	valid_0's auc: 0.556413
[817]	valid_0

...

[107]	valid_0's auc: 0.588002
[108]	valid_0's auc: 0.588022
[109]	valid_0's auc: 0.588014
[110]	valid_0's auc: 0.588006
[111]	valid_0's auc: 0.587897
[112]	valid_0's auc: 0.587959
[113]	valid_0's auc: 0.588012
[114]	valid_0's auc: 0.588135
[115]	valid_0's auc: 0.588101
[116]	valid_0's auc: 0.588003
[117]	valid_0's auc: 0.587978
[118]	valid_0's auc: 0.587929
[119]	valid_0's auc: 0.587994
[120]	valid_0's auc: 0.587948
[121]	valid_0's auc: 0.587801
[122]	valid_0's auc: 0.587758
[123]	valid_0's auc: 0.587889
[124]	valid_0's auc: 0.587946
[125]	valid_0's auc: 0.58796
[126]	valid_0's auc: 0.587937
[127]	valid_0's auc: 0.587949
[128]	valid_0's auc: 0.587966
[129]	valid_0's auc: 0.587651
[130]	valid_0's auc: 0.587702
[131]	valid_0's auc: 0.587658
[132]	valid_0's auc: 0.587546
[133]	valid_0's auc: 0.587462
[134]	valid_0's auc: 0.587692
[135]	valid_0's auc: 0.587816
[136]	valid_0's auc: 0.587722
[137]	valid_0's auc: 0.587626
[138]	valid_0's auc: 0.587409
[139]	valid_0's auc: 0.58737
[140]	valid_

[381]	valid_0's auc: 0.589482
[382]	valid_0's auc: 0.589459
[383]	valid_0's auc: 0.589418
[384]	valid_0's auc: 0.589458
[385]	valid_0's auc: 0.589539
[386]	valid_0's auc: 0.589525
[387]	valid_0's auc: 0.589552
[388]	valid_0's auc: 0.58952
[389]	valid_0's auc: 0.589557
[390]	valid_0's auc: 0.589567
[391]	valid_0's auc: 0.589527
[392]	valid_0's auc: 0.589578
[393]	valid_0's auc: 0.589632
[394]	valid_0's auc: 0.589618
[395]	valid_0's auc: 0.589552
[396]	valid_0's auc: 0.589585
[397]	valid_0's auc: 0.589638
[398]	valid_0's auc: 0.589602
[399]	valid_0's auc: 0.58956
[400]	valid_0's auc: 0.589525
[401]	valid_0's auc: 0.589537
[402]	valid_0's auc: 0.589538
[403]	valid_0's auc: 0.589576
[404]	valid_0's auc: 0.589549
[405]	valid_0's auc: 0.589565
[406]	valid_0's auc: 0.589652
[407]	valid_0's auc: 0.589655
[408]	valid_0's auc: 0.589638
[409]	valid_0's auc: 0.589729
[410]	valid_0's auc: 0.589738
[411]	valid_0's auc: 0.589726
[412]	valid_0's auc: 0.589766
[413]	valid_0's auc: 0.589717
[414]	valid_

[259]	valid_0's auc: 0.570128
[260]	valid_0's auc: 0.570134
[261]	valid_0's auc: 0.570166
[262]	valid_0's auc: 0.570174
[263]	valid_0's auc: 0.570067
[264]	valid_0's auc: 0.570155
[265]	valid_0's auc: 0.570199
[266]	valid_0's auc: 0.570164
[267]	valid_0's auc: 0.570235
[268]	valid_0's auc: 0.570296
[269]	valid_0's auc: 0.570222
[270]	valid_0's auc: 0.570249
[271]	valid_0's auc: 0.570288
[272]	valid_0's auc: 0.570225
[273]	valid_0's auc: 0.570178
[274]	valid_0's auc: 0.570343
[275]	valid_0's auc: 0.570393
[276]	valid_0's auc: 0.570348
[277]	valid_0's auc: 0.570312
[278]	valid_0's auc: 0.57031
[279]	valid_0's auc: 0.570281
[280]	valid_0's auc: 0.570339
[281]	valid_0's auc: 0.570317
[282]	valid_0's auc: 0.570435
[283]	valid_0's auc: 0.570519
[284]	valid_0's auc: 0.570393
[285]	valid_0's auc: 0.570305
[286]	valid_0's auc: 0.570348
[287]	valid_0's auc: 0.570403
[288]	valid_0's auc: 0.570368
[289]	valid_0's auc: 0.570421
[290]	valid_0's auc: 0.570375
[291]	valid_0's auc: 0.570385
[292]	valid

[805]	valid_0's auc: 0.572625
[806]	valid_0's auc: 0.572606
[807]	valid_0's auc: 0.572535
[808]	valid_0's auc: 0.572546
[809]	valid_0's auc: 0.572524
[810]	valid_0's auc: 0.572494
[811]	valid_0's auc: 0.572417
[812]	valid_0's auc: 0.572417
[813]	valid_0's auc: 0.572356
[814]	valid_0's auc: 0.572374
[815]	valid_0's auc: 0.572374
[816]	valid_0's auc: 0.572418
[817]	valid_0's auc: 0.572409
[818]	valid_0's auc: 0.572429
[819]	valid_0's auc: 0.572414
[820]	valid_0's auc: 0.572352
[821]	valid_0's auc: 0.572392
[822]	valid_0's auc: 0.572396
[823]	valid_0's auc: 0.572402
[824]	valid_0's auc: 0.572415
[825]	valid_0's auc: 0.572444
[826]	valid_0's auc: 0.572443
[827]	valid_0's auc: 0.572429
[828]	valid_0's auc: 0.572449
[829]	valid_0's auc: 0.572459
[830]	valid_0's auc: 0.572452
[831]	valid_0's auc: 0.572426
[832]	valid_0's auc: 0.572478
[833]	valid_0's auc: 0.572477
[834]	valid_0's auc: 0.572442
[835]	valid_0's auc: 0.572448
[836]	valid_0's auc: 0.572405
[837]	valid_0's auc: 0.572404
[838]	vali

[342]	valid_0's auc: 0.574306
[343]	valid_0's auc: 0.574376
[344]	valid_0's auc: 0.574422
[345]	valid_0's auc: 0.574416
[346]	valid_0's auc: 0.57453
[347]	valid_0's auc: 0.574536
[348]	valid_0's auc: 0.574618
[349]	valid_0's auc: 0.574549
[350]	valid_0's auc: 0.574584
[351]	valid_0's auc: 0.574663
[352]	valid_0's auc: 0.574651
[353]	valid_0's auc: 0.574614
[354]	valid_0's auc: 0.574645
[355]	valid_0's auc: 0.574611
[356]	valid_0's auc: 0.57464
[357]	valid_0's auc: 0.574583
[358]	valid_0's auc: 0.57455
[359]	valid_0's auc: 0.574578
[360]	valid_0's auc: 0.574508
[361]	valid_0's auc: 0.574551
[362]	valid_0's auc: 0.574603
[363]	valid_0's auc: 0.574644
[364]	valid_0's auc: 0.574591
[365]	valid_0's auc: 0.574591
[366]	valid_0's auc: 0.574564
[367]	valid_0's auc: 0.574667
[368]	valid_0's auc: 0.574681
[369]	valid_0's auc: 0.574671
[370]	valid_0's auc: 0.57462
[371]	valid_0's auc: 0.57463
[372]	valid_0's auc: 0.574657
[373]	valid_0's auc: 0.574664
[374]	valid_0's auc: 0.574645
[375]	valid_0's

In [11]:
final_test_preds=[]
final_valid_preds={}
scores=[]

for fold in range(10):
    xtrain=df[df.kfold != fold].reset_index(drop=True)
    xvalid=df[df.kfold == fold].reset_index(drop=True)
    xtest=test.copy()
    
    valid_id=xvalid.id.values.tolist()
    
    ytrain=xtrain.song_popularity
    yvalid=xvalid.song_popularity
    
    xtrain=xtrain[useful_features]
    xvalid=xvalid[useful_features]

    
    model = lgb.LGBMClassifier(**lgbm_params1)
    model.fit(xtrain, ytrain,eval_set=[(xvalid, yvalid)])
    
    preds_valid=model.predict_proba(xvalid)[:,1]
    preds_test=model.predict_proba(xtest[useful_features])[:,1]
    
    
    final_test_preds.append(preds_test)
    final_valid_preds.update(dict(zip(valid_id,preds_valid)))
    
    roc1= roc_auc_score(yvalid,preds_valid)
    #Score 
    scores.append(roc1)
    print(f"fold|split:{fold},roc:{roc1}")
    
    
print(np.mean(scores))
final_valid_pred1=pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
final_valid_pred1.columns=['id',"preds_6"]
final_valid_pred1.to_csv('train_pred_6.csv',index=False)
    
y=np.mean(np.column_stack(final_test_preds),axis=1)
test_preds1=pd.DataFrame(y,columns=['test_preds6'])
test_preds1['id']=test_preds1.index
test_preds1.columns=['preds_6','id']
test_preds1.to_csv("test_preds6.csv",index=False)
    

[1]	valid_0's auc: 0.538859
[2]	valid_0's auc: 0.543476
[3]	valid_0's auc: 0.550498
[4]	valid_0's auc: 0.555607
[5]	valid_0's auc: 0.557638
[6]	valid_0's auc: 0.561087
[7]	valid_0's auc: 0.560331
[8]	valid_0's auc: 0.559929
[9]	valid_0's auc: 0.55934
[10]	valid_0's auc: 0.558686
[11]	valid_0's auc: 0.558882
[12]	valid_0's auc: 0.559159
[13]	valid_0's auc: 0.559128
[14]	valid_0's auc: 0.560141
[15]	valid_0's auc: 0.560834
[16]	valid_0's auc: 0.561865
[17]	valid_0's auc: 0.563063
[18]	valid_0's auc: 0.564539
[19]	valid_0's auc: 0.564494
[20]	valid_0's auc: 0.564772
[21]	valid_0's auc: 0.563713
[22]	valid_0's auc: 0.562867
[23]	valid_0's auc: 0.562987
[24]	valid_0's auc: 0.562793
[25]	valid_0's auc: 0.562458
[26]	valid_0's auc: 0.562123
[27]	valid_0's auc: 0.562427
[28]	valid_0's auc: 0.56292
[29]	valid_0's auc: 0.562968
[30]	valid_0's auc: 0.562384
[31]	valid_0's auc: 0.563096
[32]	valid_0's auc: 0.562725
[33]	valid_0's auc: 0.562584
[34]	valid_0's auc: 0.562919
[35]	valid_0's auc: 0.563

[552]	valid_0's auc: 0.576288
[553]	valid_0's auc: 0.576304
[554]	valid_0's auc: 0.576297
[555]	valid_0's auc: 0.576322
[556]	valid_0's auc: 0.576324
[557]	valid_0's auc: 0.576411
[558]	valid_0's auc: 0.576437
[559]	valid_0's auc: 0.576447
[560]	valid_0's auc: 0.576434
[561]	valid_0's auc: 0.576425
[562]	valid_0's auc: 0.576438
[563]	valid_0's auc: 0.576431
[564]	valid_0's auc: 0.576497
[565]	valid_0's auc: 0.576508
[566]	valid_0's auc: 0.576494
[567]	valid_0's auc: 0.576558
[568]	valid_0's auc: 0.576554
[569]	valid_0's auc: 0.57658
[570]	valid_0's auc: 0.576585
[571]	valid_0's auc: 0.576543
[572]	valid_0's auc: 0.576587
[573]	valid_0's auc: 0.576647
[574]	valid_0's auc: 0.576704
[575]	valid_0's auc: 0.57668
[576]	valid_0's auc: 0.576676
[577]	valid_0's auc: 0.576702
[578]	valid_0's auc: 0.57671
[579]	valid_0's auc: 0.576734
[580]	valid_0's auc: 0.576768
[581]	valid_0's auc: 0.576765
[582]	valid_0's auc: 0.576821
[583]	valid_0's auc: 0.576798
[584]	valid_0's auc: 0.576767
[585]	valid_0

[1096]	valid_0's auc: 0.582672
[1097]	valid_0's auc: 0.582702
[1098]	valid_0's auc: 0.582726
[1099]	valid_0's auc: 0.582751
[1100]	valid_0's auc: 0.582748
[1101]	valid_0's auc: 0.582731
[1102]	valid_0's auc: 0.582719
[1103]	valid_0's auc: 0.58273
[1104]	valid_0's auc: 0.582752
[1105]	valid_0's auc: 0.582735
[1106]	valid_0's auc: 0.582802
[1107]	valid_0's auc: 0.582824
[1108]	valid_0's auc: 0.582797
[1109]	valid_0's auc: 0.582816
[1110]	valid_0's auc: 0.582789
[1111]	valid_0's auc: 0.582803
[1112]	valid_0's auc: 0.582775
[1113]	valid_0's auc: 0.582804
[1114]	valid_0's auc: 0.582759
[1115]	valid_0's auc: 0.582744
[1116]	valid_0's auc: 0.582758
[1117]	valid_0's auc: 0.582742
[1118]	valid_0's auc: 0.582759
[1119]	valid_0's auc: 0.582764
[1120]	valid_0's auc: 0.582784
[1121]	valid_0's auc: 0.58282
[1122]	valid_0's auc: 0.582847
[1123]	valid_0's auc: 0.58286
[1124]	valid_0's auc: 0.58288
[1125]	valid_0's auc: 0.582882
[1126]	valid_0's auc: 0.582873
[1127]	valid_0's auc: 0.582905
[1128]	valid

[1626]	valid_0's auc: 0.585272
[1627]	valid_0's auc: 0.585238
[1628]	valid_0's auc: 0.58525
[1629]	valid_0's auc: 0.585252
[1630]	valid_0's auc: 0.585296
[1631]	valid_0's auc: 0.585301
[1632]	valid_0's auc: 0.585305
[1633]	valid_0's auc: 0.585315
[1634]	valid_0's auc: 0.585362
[1635]	valid_0's auc: 0.585367
[1636]	valid_0's auc: 0.585393
[1637]	valid_0's auc: 0.585374
[1638]	valid_0's auc: 0.585413
[1639]	valid_0's auc: 0.585399
[1640]	valid_0's auc: 0.585424
[1641]	valid_0's auc: 0.585413
[1642]	valid_0's auc: 0.585416
[1643]	valid_0's auc: 0.585409
[1644]	valid_0's auc: 0.585449
[1645]	valid_0's auc: 0.585454
[1646]	valid_0's auc: 0.585453
[1647]	valid_0's auc: 0.585418
[1648]	valid_0's auc: 0.585433
[1649]	valid_0's auc: 0.585414
[1650]	valid_0's auc: 0.585408
[1651]	valid_0's auc: 0.585392
[1652]	valid_0's auc: 0.585451
[1653]	valid_0's auc: 0.585463
[1654]	valid_0's auc: 0.585468
[1655]	valid_0's auc: 0.585454
[1656]	valid_0's auc: 0.585449
[1657]	valid_0's auc: 0.585469
[1658]	va

[2156]	valid_0's auc: 0.586008
[2157]	valid_0's auc: 0.586002
[2158]	valid_0's auc: 0.586016
[2159]	valid_0's auc: 0.586044
[2160]	valid_0's auc: 0.586074
[2161]	valid_0's auc: 0.586103
[2162]	valid_0's auc: 0.586111
[2163]	valid_0's auc: 0.586133
[2164]	valid_0's auc: 0.586134
[2165]	valid_0's auc: 0.586122
[2166]	valid_0's auc: 0.58615
[2167]	valid_0's auc: 0.586165
[2168]	valid_0's auc: 0.586195
[2169]	valid_0's auc: 0.586183
[2170]	valid_0's auc: 0.586174
[2171]	valid_0's auc: 0.586146
[2172]	valid_0's auc: 0.586144
[2173]	valid_0's auc: 0.586179
[2174]	valid_0's auc: 0.586171
[2175]	valid_0's auc: 0.586166
[2176]	valid_0's auc: 0.586131
[2177]	valid_0's auc: 0.586126
[2178]	valid_0's auc: 0.586136
[2179]	valid_0's auc: 0.58612
[2180]	valid_0's auc: 0.586139
[2181]	valid_0's auc: 0.586127
[2182]	valid_0's auc: 0.586109
[2183]	valid_0's auc: 0.586116
[2184]	valid_0's auc: 0.586125
[2185]	valid_0's auc: 0.586132
[2186]	valid_0's auc: 0.58615
[2187]	valid_0's auc: 0.586103
[2188]	vali

...

[647]	valid_0's auc: 0.568492
[648]	valid_0's auc: 0.568513
[649]	valid_0's auc: 0.568485
[650]	valid_0's auc: 0.568483
[651]	valid_0's auc: 0.568536
[652]	valid_0's auc: 0.568563
[653]	valid_0's auc: 0.568528
[654]	valid_0's auc: 0.56852
[655]	valid_0's auc: 0.568578
[656]	valid_0's auc: 0.568575
[657]	valid_0's auc: 0.568615
[658]	valid_0's auc: 0.56867
[659]	valid_0's auc: 0.568705
[660]	valid_0's auc: 0.568688
[661]	valid_0's auc: 0.568681
[662]	valid_0's auc: 0.568728
[663]	valid_0's auc: 0.568756
[664]	valid_0's auc: 0.568712
[665]	valid_0's auc: 0.56871
[666]	valid_0's auc: 0.568697
[667]	valid_0's auc: 0.568693
[668]	valid_0's auc: 0.568668
[669]	valid_0's auc: 0.568625
[670]	valid_0's auc: 0.568654
[671]	valid_0's auc: 0.568669
[672]	valid_0's auc: 0.56869
[673]	valid_0's auc: 0.568669
[674]	valid_0's auc: 0.568655
[675]	valid_0's auc: 0.568631
[676]	valid_0's auc: 0.568671
[677]	valid_0's auc: 0.568653
[678]	valid_0's auc: 0.568653
[679]	valid_0's auc: 0.568635
[680]	valid_0'

[1187]	valid_0's auc: 0.571779
[1188]	valid_0's auc: 0.571761
[1189]	valid_0's auc: 0.571775
[1190]	valid_0's auc: 0.57177
[1191]	valid_0's auc: 0.571708
[1192]	valid_0's auc: 0.57174
[1193]	valid_0's auc: 0.571748
[1194]	valid_0's auc: 0.571737
[1195]	valid_0's auc: 0.571744
[1196]	valid_0's auc: 0.571729
[1197]	valid_0's auc: 0.571751
[1198]	valid_0's auc: 0.571778
[1199]	valid_0's auc: 0.571766
[1200]	valid_0's auc: 0.571767
[1201]	valid_0's auc: 0.571776
[1202]	valid_0's auc: 0.571785
[1203]	valid_0's auc: 0.571766
[1204]	valid_0's auc: 0.57177
[1205]	valid_0's auc: 0.571792
[1206]	valid_0's auc: 0.571783
[1207]	valid_0's auc: 0.571798
[1208]	valid_0's auc: 0.571827
[1209]	valid_0's auc: 0.571844
[1210]	valid_0's auc: 0.571854
[1211]	valid_0's auc: 0.571828
[1212]	valid_0's auc: 0.571806
[1213]	valid_0's auc: 0.571848
[1214]	valid_0's auc: 0.571827
[1215]	valid_0's auc: 0.571835
[1216]	valid_0's auc: 0.571845
[1217]	valid_0's auc: 0.571818
[1218]	valid_0's auc: 0.571804
[1219]	vali

[1717]	valid_0's auc: 0.572741
[1718]	valid_0's auc: 0.572766
[1719]	valid_0's auc: 0.572783
[1720]	valid_0's auc: 0.572797
[1721]	valid_0's auc: 0.572816
[1722]	valid_0's auc: 0.572796
[1723]	valid_0's auc: 0.572786
[1724]	valid_0's auc: 0.572778
[1725]	valid_0's auc: 0.572755
[1726]	valid_0's auc: 0.572743
[1727]	valid_0's auc: 0.572746
[1728]	valid_0's auc: 0.57278
[1729]	valid_0's auc: 0.572781
[1730]	valid_0's auc: 0.572774
[1731]	valid_0's auc: 0.57277
[1732]	valid_0's auc: 0.572789
[1733]	valid_0's auc: 0.572799
[1734]	valid_0's auc: 0.572774
[1735]	valid_0's auc: 0.572753
[1736]	valid_0's auc: 0.572748
[1737]	valid_0's auc: 0.572772
[1738]	valid_0's auc: 0.572752
[1739]	valid_0's auc: 0.572722
[1740]	valid_0's auc: 0.572712
[1741]	valid_0's auc: 0.572721
[1742]	valid_0's auc: 0.572732
[1743]	valid_0's auc: 0.572718
[1744]	valid_0's auc: 0.572718
[1745]	valid_0's auc: 0.572716
[1746]	valid_0's auc: 0.572743
[1747]	valid_0's auc: 0.57278
[1748]	valid_0's auc: 0.572799
[1749]	vali

[2247]	valid_0's auc: 0.572983
[2248]	valid_0's auc: 0.572924
[2249]	valid_0's auc: 0.57293
[2250]	valid_0's auc: 0.572936
[2251]	valid_0's auc: 0.572924
[2252]	valid_0's auc: 0.572935
[2253]	valid_0's auc: 0.572924
[2254]	valid_0's auc: 0.572888
[2255]	valid_0's auc: 0.572909
[2256]	valid_0's auc: 0.572918
[2257]	valid_0's auc: 0.572884
[2258]	valid_0's auc: 0.572891
[2259]	valid_0's auc: 0.572884
[2260]	valid_0's auc: 0.572903
[2261]	valid_0's auc: 0.572898
[2262]	valid_0's auc: 0.572904
[2263]	valid_0's auc: 0.572906
[2264]	valid_0's auc: 0.572901
[2265]	valid_0's auc: 0.572938
[2266]	valid_0's auc: 0.572923
[2267]	valid_0's auc: 0.572917
[2268]	valid_0's auc: 0.572928
[2269]	valid_0's auc: 0.572928
[2270]	valid_0's auc: 0.572951
[2271]	valid_0's auc: 0.572966
[2272]	valid_0's auc: 0.572968
[2273]	valid_0's auc: 0.572957
[2274]	valid_0's auc: 0.57296
[2275]	valid_0's auc: 0.572969
[2276]	valid_0's auc: 0.572967
[2277]	valid_0's auc: 0.572977
[2278]	valid_0's auc: 0.572973
[2279]	val

[491]	valid_0's auc: 0.575633
[492]	valid_0's auc: 0.575565
[493]	valid_0's auc: 0.575575
[494]	valid_0's auc: 0.575554
[495]	valid_0's auc: 0.575549
[496]	valid_0's auc: 0.575516
[497]	valid_0's auc: 0.575507
[498]	valid_0's auc: 0.575472
[499]	valid_0's auc: 0.575479
[500]	valid_0's auc: 0.575527
[501]	valid_0's auc: 0.575492
[502]	valid_0's auc: 0.575493
[503]	valid_0's auc: 0.575417
[504]	valid_0's auc: 0.575439
[505]	valid_0's auc: 0.575441
[506]	valid_0's auc: 0.575446
[507]	valid_0's auc: 0.575498
[508]	valid_0's auc: 0.575544
[509]	valid_0's auc: 0.575655
[510]	valid_0's auc: 0.575657
[511]	valid_0's auc: 0.575622
[512]	valid_0's auc: 0.57567
[513]	valid_0's auc: 0.57567
[514]	valid_0's auc: 0.575688
[515]	valid_0's auc: 0.575709
[516]	valid_0's auc: 0.575678
[517]	valid_0's auc: 0.575671
[518]	valid_0's auc: 0.575652
[519]	valid_0's auc: 0.575693
[520]	valid_0's auc: 0.575696
[521]	valid_0's auc: 0.575749
[522]	valid_0's auc: 0.575821
[523]	valid_0's auc: 0.575802
[524]	valid_

In [14]:
# final_test_preds=[]
# final_valid_preds={}
# scores=[]

# for fold in range(10):
#     xtrain=df[df.kfold != fold].reset_index(drop=True)
#     xvalid=df[df.kfold == fold].reset_index(drop=True)
#     xtest=test.copy()
    
#     valid_id=xvalid.id.values.tolist()
    
#     ytrain=xtrain.song_popularity
#     yvalid=xvalid.song_popularity
    
#     xtrain=xtrain[useful_features]
#     xvalid=xvalid[useful_features]

    
#     model = lgb.LGBMClassifier(**lgbm_params2)
#     model.fit(xtrain, ytrain,eval_set=[(xvalid, yvalid)])
    
#     preds_valid=model.predict_proba(xvalid)[:,1]
#     preds_test=model.predict_proba(xtest[useful_features])[:,1]
    
    
#     final_test_preds.append(preds_test)
#     final_valid_preds.update(dict(zip(valid_id,preds_valid)))
    
#     roc1= roc_auc_score(yvalid,preds_valid)
#     #Score 
#     scores.append(roc1)
#     print(f"fold|split:{fold},roc:{roc1}")
    
    
# print(np.mean(scores))
# final_valid_pred1=pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
# final_valid_pred1.columns=['id',"preds_3"]
# final_valid_pred1.to_csv('train_pred_3.csv',index=False)
    
# y=np.mean(np.column_stack(final_test_preds),axis=1)
# test_preds1=pd.DataFrame(y,columns=['test_preds3'])
# test_preds1['id']=test_preds1.index
# test_preds1.columns=['preds_3','id']
# test_preds1.to_csv("test_preds3.csv",index=False)
    

[1]	valid_0's auc: 0.54164
[2]	valid_0's auc: 0.552719
[3]	valid_0's auc: 0.552429
[4]	valid_0's auc: 0.551602
[5]	valid_0's auc: 0.552448
[6]	valid_0's auc: 0.555393
[7]	valid_0's auc: 0.553876
[8]	valid_0's auc: 0.553035
[9]	valid_0's auc: 0.553546
[10]	valid_0's auc: 0.553871
[11]	valid_0's auc: 0.552464
[12]	valid_0's auc: 0.553417
[13]	valid_0's auc: 0.55468
[14]	valid_0's auc: 0.555212
[15]	valid_0's auc: 0.555589
[16]	valid_0's auc: 0.555853
[17]	valid_0's auc: 0.555748
[18]	valid_0's auc: 0.555758
[19]	valid_0's auc: 0.556333
[20]	valid_0's auc: 0.555948
[21]	valid_0's auc: 0.556297
[22]	valid_0's auc: 0.556529
[23]	valid_0's auc: 0.556301
[24]	valid_0's auc: 0.556563
[25]	valid_0's auc: 0.556424
[26]	valid_0's auc: 0.556691
[27]	valid_0's auc: 0.556605
[28]	valid_0's auc: 0.555893
[29]	valid_0's auc: 0.55575
[30]	valid_0's auc: 0.555093
[31]	valid_0's auc: 0.555317
[32]	valid_0's auc: 0.555331
[33]	valid_0's auc: 0.555595
[34]	valid_0's auc: 0.555742
[35]	valid_0's auc: 0.5554

[552]	valid_0's auc: 0.558571
[553]	valid_0's auc: 0.558568
[554]	valid_0's auc: 0.558534
[555]	valid_0's auc: 0.558592
[556]	valid_0's auc: 0.558656
[557]	valid_0's auc: 0.558699
[558]	valid_0's auc: 0.558695
[559]	valid_0's auc: 0.558655
[560]	valid_0's auc: 0.558654
[561]	valid_0's auc: 0.558675
[562]	valid_0's auc: 0.558704
[563]	valid_0's auc: 0.558717
[564]	valid_0's auc: 0.55873
[565]	valid_0's auc: 0.558726
[566]	valid_0's auc: 0.558786
[567]	valid_0's auc: 0.558831
[568]	valid_0's auc: 0.558859
[569]	valid_0's auc: 0.558892
[570]	valid_0's auc: 0.558928
[571]	valid_0's auc: 0.558917
[572]	valid_0's auc: 0.558967
[573]	valid_0's auc: 0.558997
[574]	valid_0's auc: 0.558994
[575]	valid_0's auc: 0.55899
[576]	valid_0's auc: 0.558953
[577]	valid_0's auc: 0.558946
[578]	valid_0's auc: 0.558953
[579]	valid_0's auc: 0.558951
[580]	valid_0's auc: 0.558986
[581]	valid_0's auc: 0.559018
[582]	valid_0's auc: 0.559019
[583]	valid_0's auc: 0.559059
[584]	valid_0's auc: 0.559057
[585]	valid_

[826]	valid_0's auc: 0.560946
[827]	valid_0's auc: 0.560933
[828]	valid_0's auc: 0.560925
[829]	valid_0's auc: 0.560934
[830]	valid_0's auc: 0.560908
[831]	valid_0's auc: 0.560931
[832]	valid_0's auc: 0.560938
[833]	valid_0's auc: 0.56097
[834]	valid_0's auc: 0.560983
[835]	valid_0's auc: 0.560988
[836]	valid_0's auc: 0.56103
[837]	valid_0's auc: 0.561056
[838]	valid_0's auc: 0.561031
[839]	valid_0's auc: 0.561017
[840]	valid_0's auc: 0.561019
[841]	valid_0's auc: 0.561042
[842]	valid_0's auc: 0.561051
[843]	valid_0's auc: 0.561062
[844]	valid_0's auc: 0.56108
[845]	valid_0's auc: 0.561097
[846]	valid_0's auc: 0.561063
[847]	valid_0's auc: 0.561046
[848]	valid_0's auc: 0.561076
[849]	valid_0's auc: 0.561078
[850]	valid_0's auc: 0.561078
[851]	valid_0's auc: 0.561095
[852]	valid_0's auc: 0.561111
[853]	valid_0's auc: 0.561132
[854]	valid_0's auc: 0.561144
[855]	valid_0's auc: 0.561158
[856]	valid_0's auc: 0.561182
[857]	valid_0's auc: 0.56122
[858]	valid_0's auc: 0.561211
[859]	valid_0'

[1361]	valid_0's auc: 0.564979
[1362]	valid_0's auc: 0.564963
[1363]	valid_0's auc: 0.564961
[1364]	valid_0's auc: 0.56497
[1365]	valid_0's auc: 0.564995
[1366]	valid_0's auc: 0.565018
[1367]	valid_0's auc: 0.565019
[1368]	valid_0's auc: 0.565033
[1369]	valid_0's auc: 0.565011
[1370]	valid_0's auc: 0.565022
[1371]	valid_0's auc: 0.565044
[1372]	valid_0's auc: 0.565067
[1373]	valid_0's auc: 0.565058
[1374]	valid_0's auc: 0.565061
[1375]	valid_0's auc: 0.565088
[1376]	valid_0's auc: 0.56507
[1377]	valid_0's auc: 0.56508
[1378]	valid_0's auc: 0.565084
[1379]	valid_0's auc: 0.565104
[1380]	valid_0's auc: 0.565101
[1381]	valid_0's auc: 0.565107
[1382]	valid_0's auc: 0.56512
[1383]	valid_0's auc: 0.565126
[1384]	valid_0's auc: 0.565126
[1385]	valid_0's auc: 0.565162
[1386]	valid_0's auc: 0.565176
[1387]	valid_0's auc: 0.565181
[1388]	valid_0's auc: 0.565189
[1389]	valid_0's auc: 0.565196
[1390]	valid_0's auc: 0.565178
[1391]	valid_0's auc: 0.565214
[1392]	valid_0's auc: 0.565222
[1393]	valid

[1891]	valid_0's auc: 0.56841
[1892]	valid_0's auc: 0.568407
[1893]	valid_0's auc: 0.568436
[1894]	valid_0's auc: 0.568434
[1895]	valid_0's auc: 0.56844
[1896]	valid_0's auc: 0.568433
[1897]	valid_0's auc: 0.568446
[1898]	valid_0's auc: 0.568452
[1899]	valid_0's auc: 0.568453
[1900]	valid_0's auc: 0.568471
[1901]	valid_0's auc: 0.56848
[1902]	valid_0's auc: 0.568489
[1903]	valid_0's auc: 0.568498
[1904]	valid_0's auc: 0.568496
[1905]	valid_0's auc: 0.56849
[1906]	valid_0's auc: 0.568497
[1907]	valid_0's auc: 0.568513
[1908]	valid_0's auc: 0.568499
[1909]	valid_0's auc: 0.568497
[1910]	valid_0's auc: 0.568495
[1911]	valid_0's auc: 0.568509
[1912]	valid_0's auc: 0.568519
[1913]	valid_0's auc: 0.568525
[1914]	valid_0's auc: 0.568518
[1915]	valid_0's auc: 0.568547
[1916]	valid_0's auc: 0.568538
[1917]	valid_0's auc: 0.56853
[1918]	valid_0's auc: 0.568549
[1919]	valid_0's auc: 0.568557
[1920]	valid_0's auc: 0.568563
[1921]	valid_0's auc: 0.568566
[1922]	valid_0's auc: 0.568578
[1923]	valid_

...

[6353]	valid_0's auc: 0.566689
[6354]	valid_0's auc: 0.566687
[6355]	valid_0's auc: 0.566692
[6356]	valid_0's auc: 0.566696
[6357]	valid_0's auc: 0.566699
[6358]	valid_0's auc: 0.566699
[6359]	valid_0's auc: 0.566703
[6360]	valid_0's auc: 0.566697
[6361]	valid_0's auc: 0.566697
[6362]	valid_0's auc: 0.566699
[6363]	valid_0's auc: 0.566693
[6364]	valid_0's auc: 0.566695
[6365]	valid_0's auc: 0.566698
[6366]	valid_0's auc: 0.566692
[6367]	valid_0's auc: 0.566696
[6368]	valid_0's auc: 0.566695
[6369]	valid_0's auc: 0.566694
[6370]	valid_0's auc: 0.56669
[6371]	valid_0's auc: 0.566693
[6372]	valid_0's auc: 0.566693
[6373]	valid_0's auc: 0.566693
[6374]	valid_0's auc: 0.566687
[6375]	valid_0's auc: 0.566678
[6376]	valid_0's auc: 0.56668
[6377]	valid_0's auc: 0.566685
[6378]	valid_0's auc: 0.566693
[6379]	valid_0's auc: 0.566697
[6380]	valid_0's auc: 0.566703
[6381]	valid_0's auc: 0.566694
[6382]	valid_0's auc: 0.56669
[6383]	valid_0's auc: 0.566679
[6384]	valid_0's auc: 0.566684
[6385]	vali

[6883]	valid_0's auc: 0.566898
[6884]	valid_0's auc: 0.566896
[6885]	valid_0's auc: 0.566902
[6886]	valid_0's auc: 0.566888
[6887]	valid_0's auc: 0.566899
[6888]	valid_0's auc: 0.566895
[6889]	valid_0's auc: 0.56689
[6890]	valid_0's auc: 0.566888
[6891]	valid_0's auc: 0.566882
[6892]	valid_0's auc: 0.566889
[6893]	valid_0's auc: 0.566895
[6894]	valid_0's auc: 0.566903
[6895]	valid_0's auc: 0.566897
[6896]	valid_0's auc: 0.566894
[6897]	valid_0's auc: 0.566887
[6898]	valid_0's auc: 0.566893
[6899]	valid_0's auc: 0.566888
[6900]	valid_0's auc: 0.566895
[6901]	valid_0's auc: 0.566895
[6902]	valid_0's auc: 0.566901
[6903]	valid_0's auc: 0.566895
[6904]	valid_0's auc: 0.566899
[6905]	valid_0's auc: 0.566893
[6906]	valid_0's auc: 0.566906
[6907]	valid_0's auc: 0.566908
[6908]	valid_0's auc: 0.566903
[6909]	valid_0's auc: 0.566907
[6910]	valid_0's auc: 0.566905
[6911]	valid_0's auc: 0.566902
[6912]	valid_0's auc: 0.566905
[6913]	valid_0's auc: 0.566904
[6914]	valid_0's auc: 0.566899
[6915]	va

[7413]	valid_0's auc: 0.567053
[7414]	valid_0's auc: 0.567056
[7415]	valid_0's auc: 0.567058
[7416]	valid_0's auc: 0.567064
[7417]	valid_0's auc: 0.567066
[7418]	valid_0's auc: 0.567066
[7419]	valid_0's auc: 0.567065
[7420]	valid_0's auc: 0.567074
[7421]	valid_0's auc: 0.56707
[7422]	valid_0's auc: 0.567067
[7423]	valid_0's auc: 0.56706
[7424]	valid_0's auc: 0.567069
[7425]	valid_0's auc: 0.567063
[7426]	valid_0's auc: 0.567058
[7427]	valid_0's auc: 0.567061
[7428]	valid_0's auc: 0.567056
[7429]	valid_0's auc: 0.567056
[7430]	valid_0's auc: 0.567068
[7431]	valid_0's auc: 0.567062
[7432]	valid_0's auc: 0.567066
[7433]	valid_0's auc: 0.567067
[7434]	valid_0's auc: 0.567072
[7435]	valid_0's auc: 0.567073
[7436]	valid_0's auc: 0.567068
[7437]	valid_0's auc: 0.567067
[7438]	valid_0's auc: 0.567062
[7439]	valid_0's auc: 0.567054
[7440]	valid_0's auc: 0.567062
[7441]	valid_0's auc: 0.567062
[7442]	valid_0's auc: 0.567064
[7443]	valid_0's auc: 0.567065
[7444]	valid_0's auc: 0.567065
[7445]	val

[7943]	valid_0's auc: 0.567308
[7944]	valid_0's auc: 0.567315
[7945]	valid_0's auc: 0.567312
[7946]	valid_0's auc: 0.567307
[7947]	valid_0's auc: 0.567308
[7948]	valid_0's auc: 0.567313
[7949]	valid_0's auc: 0.567316
[7950]	valid_0's auc: 0.567316
[7951]	valid_0's auc: 0.567318
[7952]	valid_0's auc: 0.567317
[7953]	valid_0's auc: 0.567322
[7954]	valid_0's auc: 0.567323
[7955]	valid_0's auc: 0.567335
[7956]	valid_0's auc: 0.56734
[7957]	valid_0's auc: 0.567338
[7958]	valid_0's auc: 0.567345
[7959]	valid_0's auc: 0.567334
[7960]	valid_0's auc: 0.567338
[7961]	valid_0's auc: 0.567342
[7962]	valid_0's auc: 0.567344
[7963]	valid_0's auc: 0.567339
[7964]	valid_0's auc: 0.567346
[7965]	valid_0's auc: 0.567352
[7966]	valid_0's auc: 0.56735
[7967]	valid_0's auc: 0.567353
[7968]	valid_0's auc: 0.567355
[7969]	valid_0's auc: 0.567349
[7970]	valid_0's auc: 0.567353
[7971]	valid_0's auc: 0.567349
[7972]	valid_0's auc: 0.567343
[7973]	valid_0's auc: 0.567339
[7974]	valid_0's auc: 0.56735
[7975]	vali

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	valid_0's auc: 0.546987
[2]	valid_0's auc: 0.551468
[3]	valid_0's auc: 0.552179
[4]	valid_0's auc: 0.555782
[5]	valid_0's auc: 0.559326
[6]	valid_0's auc: 0.562016
[7]	valid_0's auc: 0.562468
[8]	valid_0's auc: 0.564614
[9]	valid_0's auc: 0.565077
[10]	valid_0's auc: 0.564681
[11]	valid_0's auc: 0.564014
[12]	valid_0's auc: 0.56415
[13]	valid_0's auc: 0.563761
[14]	valid_0's auc: 0.565164
[15]	valid_0's auc: 0.564942
[16]	valid_0's auc: 0.563753
[17]	valid_0's auc: 0.563785
[18]	valid_0's auc: 0.563356
[19]	valid_0's auc: 0.562447
[20]	valid_0's auc: 0.56313
[21]	valid_0's auc: 0.563238
[22]	valid_0's auc: 0.563218
[23]	valid_0's auc: 0.563984
[24]	valid_0's auc: 0.564273
[25]	valid_0's auc: 0.56359
[26]	valid_0's auc: 0.562905
[27]	valid_0's auc: 0.562533
[28]	valid_0's auc: 0.562156
[29]	valid_0's auc: 0.562001
[30]	valid_0's auc: 0.561775
[31]	valid_0's auc: 0.561879
[32]	

## XGBOOST 

In [17]:
# final_test_preds=[]
# final_valid_preds={}
# scores=[]

# for fold in range(10):
#     xtrain=df[df.kfold != fold].reset_index(drop=True)
#     xvalid=df[df.kfold == fold].reset_index(drop=True)
#     xtest=test.copy()
    
#     valid_id=xvalid.id.values.tolist()
    
#     ytrain=xtrain.song_popularity
#     yvalid=xvalid.song_popularity
    
#     xtrain=xtrain[useful_features]
#     xvalid=xvalid[useful_features]

    
#     model = XGBClassifier(**xgb_params,random_state=21)
#     model.fit(xtrain, ytrain, early_stopping_rounds = 30, eval_metric="auc",
#                            eval_set=[(xvalid,yvalid)], verbose = True)
    
#     preds_valid=model.predict_proba(xvalid)[:,1]
#     preds_test=model.predict_proba(xtest[useful_features])[:,1]
    
    
#     final_test_preds.append(preds_test)
#     final_valid_preds.update(dict(zip(valid_id,preds_valid)))
    
#     roc1= roc_auc_score(yvalid,preds_valid)
#     #Score 
#     scores.append(roc1)
#     print(f"fold|split:{fold},roc:{roc1}")
    
    
# print(np.mean(scores))
# final_valid_pred1=pd.DataFrame.from_dict(final_valid_preds,orient='index').reset_index()
# final_valid_pred1.columns=['id',"preds_3"]
# final_valid_pred1.to_csv('train_pred_3.csv',index=False)
    
# y=np.mean(np.column_stack(final_test_preds),axis=1)
# test_preds1=pd.DataFrame(y,columns=['test_preds3'])
# test_preds1['id']=test_preds1.index
# test_preds1.columns=['preds_3','id']
# test_preds1.to_csv("test_preds3.csv",index=False)
    

# BLENDING 

In [18]:
test['id']=test.index

In [19]:
train1=pd.read_csv("train_pred_1.csv")
train2=pd.read_csv("train_pred_2.csv")
# train3=pd.read_csv("train_pred_3.csv")
train4=pd.read_csv("train_pred_4.csv")
train5=pd.read_csv("train_pred_5.csv")
train6=pd.read_csv("train_pred_6.csv")

test1=pd.read_csv("test_preds1.csv")
test2=pd.read_csv("test_preds2.csv")
# test3=pd.read_csv("test_preds3.csv")
test4=pd.read_csv("test_preds4.csv")
test5=pd.read_csv("test_preds5.csv")
test6=pd.read_csv("test_preds6.csv")


df=df.merge(train1,on="id",how="left")
df=df.merge(train2,on="id",how="left")
# df=df.merge(train3,on="id",how="left")
df=df.merge(train4,on="id",how="left")
df=df.merge(train5,on="id",how="left")
df=df.merge(train6,on="id",how="left")

test=test.merge(test1,on="id",how="left")
test=test.merge(test2,on="id",how="left")
# test=test.merge(test3,on="id",how="left")
test=test.merge(test4,on="id",how="left")
test=test.merge(test5,on="id",how="left")
test=test.merge(test6,on="id",how="left")

In [20]:
final_features=['preds_1','preds_2','preds_4','preds_5','preds_6']
target=df['song_popularity']

In [28]:
from sklearn.linear_model import  LinearRegression,LogisticRegression
    
final_model =LinearRegression()
final_model.fit(df[final_features],df['song_popularity'])

LinearRegression()

## SUBMISSION

In [29]:
final_preds=final_model.predict(test[final_features])

In [30]:
sub=sample_submission
sub.head()

,id,song_popularity
0,0,0.396832
1,1,0.436252
2,2,0.309931
3,3,0.339140
4,4,0.400138


In [31]:
sub.song_popularity=final_preds

In [32]:
sub.head()

,id,song_popularity
0,0,0.395909
1,1,0.451777
2,2,0.307035
3,3,0.328402
4,4,0.385119


In [33]:
sub.to_csv("FinallinearRegression_Submission.csv",index=False)